In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_dim_datetime 
AS
SELECT fixture_date, fixture_timezone
FROM football_fixtures_raw
GROUP BY 1, 2;

INSERT INTO cs779.dim_datetime (fixture_date, fixture_timezone)
SELECT fixture_date, fixture_timezone
FROM vw_dim_datetime
EXCEPT
SELECT fixture_date, fixture_timezone
FROM cs779.dim_datetime

num_affected_rows,num_inserted_rows
422,422


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_dim_referee 
AS
SELECT fixture_referee
FROM football_fixtures_raw
GROUP BY 1;

INSERT INTO cs779.dim_referee (fixture_referee)
SELECT fixture_referee
FROM vw_dim_referee
EXCEPT
SELECT fixture_referee
FROM cs779.dim_referee

num_affected_rows,num_inserted_rows
1394,1394


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_dim_venue 
AS
SELECT fixture_venue_name AS fixture_venue, fixture_venue_city
FROM football_fixtures_raw
GROUP BY 1, 2;

INSERT INTO cs779.dim_venue (fixture_venue, fixture_venue_city)
SELECT fixture_venue, fixture_venue_city
FROM vw_dim_venue
EXCEPT
SELECT fixture_venue, fixture_venue_city
FROM cs779.dim_venue

num_affected_rows,num_inserted_rows
3114,3114


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_dim_league
AS
SELECT league_name
FROM football_fixtures_raw
GROUP BY 1;

INSERT INTO cs779.dim_league (league_name)
SELECT league_name
FROM vw_dim_league
EXCEPT
SELECT league_name
FROM cs779.dim_league

num_affected_rows,num_inserted_rows
436,436


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_dim_team
AS
SELECT teams_home_name AS teams_name
FROM football_fixtures_raw
GROUP BY 1
UNION
SELECT teams_away_name AS teams_name
FROM football_fixtures_raw
GROUP BY 1;

INSERT INTO cs779.dim_team (teams_name)
SELECT teams_name
FROM vw_dim_team
EXCEPT
SELECT teams_name
FROM cs779.dim_team

num_affected_rows,num_inserted_rows
6104,6104


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW vw_fct_football_fixtures_english
AS
WITH 
table1 AS (
  SELECT
    fixture_id,
    fixture_date,
    fixture_referee,
    fixture_venue_name AS fixture_venue,
    league_name,
    teams_home_name,
    teams_away_name,
    goals_home,
    goals_away,
    score_halftime_home,
    score_halftime_away,
    score_fulltime_home,
    score_fulltime_away,
    teams_home_winner,
    teams_away_winner
  FROM football_fixtures_raw
  WHERE league_id IN (
    SELECT league_id
    FROM football_english_leagues
  ) AND fixture_status_long = 'Match Finished'
),
table2 AS (
  SELECT
    t1.fixture_id,
    dd.dim_date_id,
    dr.dim_referee_id,
    dv.dim_venue_id,
    dl.dim_league_id,
    dt.dim_team_id AS dim_home_team_id,
    teams_away_name,
    t1.goals_home,
    t1.goals_away,
    t1.score_halftime_home,
    t1.score_halftime_away,
    t1.score_fulltime_home,
    t1.score_fulltime_away,
    t1.teams_home_winner,
    t1.teams_away_winner
  FROM table1 AS t1
  LEFT JOIN dim_datetime AS dd
    ON t1.fixture_date = dd.fixture_date
  LEFT JOIN dim_referee AS dr
    ON t1.fixture_referee = dr.fixture_referee
  LEFT JOIN dim_venue AS dv
    ON t1.fixture_venue = dv.fixture_venue
  LEFT JOIN dim_league AS dl
    ON t1.league_name = dl.league_name
  LEFT JOIN dim_team AS dt
    ON t1.teams_home_name = dt.teams_name
),
table3 AS (
  SELECT
    t2.fixture_id,
    t2.dim_date_id,
    t2.dim_referee_id,
    t2.dim_venue_id,
    t2.dim_league_id,
    t2.dim_home_team_id,
    dt.dim_team_id AS dim_away_team_id,
    t2.goals_home,
    t2.goals_away,
    t2.score_halftime_home,
    t2.score_halftime_away,
    t2.score_fulltime_home,
    t2.score_fulltime_away,
    t2.teams_home_winner AS home_victory,
    t2.teams_away_winner AS away_victory
  FROM table2 AS t2
  LEFT JOIN dim_team AS dt
    ON t2.teams_away_name = dt. teams_name
),
table4 AS (
  SELECT
    *,
    CASE
      WHEN home_victory IS NULL AND away_victory IS NULL THEN TRUE
      ELSE FALSE
    END AS draw
  FROM table3
)
SELECT *
FROM table4;

INSERT INTO cs779.fct_football_fixtures_english (
  fixture_id, 
  dim_date_id, 
  dim_referee_id, 
  dim_venue_id, 
  dim_league_id, 
  dim_home_team_id, 
  dim_away_team_id, 
  goals_home, 
  goals_away, 
  score_halftime_home, 
  score_halftime_away, 
  score_fulltime_home, 
  score_fulltime_away, 
  home_victory, 
  away_victory, 
  draw
  )
SELECT *
FROM vw_fct_football_fixtures_english
EXCEPT
SELECT 
  fixture_id, 
  dim_date_id, 
  dim_referee_id, 
  dim_venue_id, 
  dim_league_id, 
  dim_home_team_id, 
  dim_away_team_id, 
  goals_home, 
  goals_away, 
  score_halftime_home, 
  score_halftime_away, 
  score_fulltime_home, 
  score_fulltime_away, 
  home_victory, 
  away_victory, 
  draw
FROM cs779.fct_football_fixtures_english

num_affected_rows,num_inserted_rows
349,349
